## Evaluation of prediction results

 <a target="_blank" href="https://colab.research.google.com/github/moj-analytical-services/splink/blob/splink4_dev/docs/demos/tutorials/07_Quality_assurance.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In the previous tutorial, we looked at various ways to visualise the results of our model.
These are useful for evaluating a linkage pipeline because they allow us to understand how our model works and verify that it is doing something sensible. They can also be useful to identify examples where the model is not performing as expected.

In addition to these spot checks, Splink also has functions to perform more formal accuracy analysis. These functions allow you to understand the likely prevalence of false positives and false negatives in your linkage models.

They rely on the existence of a sample of labelled (ground truth) matches, which may have been produced (for example) by human beings. For the accuracy analysis to be unbiased, the sample should be representative of the overall dataset.


In [1]:
# Uncomment and run this cell if you're running in Google Colab.
# !pip install git+https://github.com/moj-analytical-services/splink.git@splink4_dev

In [2]:
# Rerun our predictions to we're ready to view the charts
import pandas as pd

from splink import DuckDBAPI, Linker, splink_datasets

pd.options.display.max_columns = 1000

db_api = DuckDBAPI()
df = splink_datasets.fake_1000

In [3]:
import json
import urllib

from splink import block_on

url = "https://raw.githubusercontent.com/moj-analytical-services/splink/479cef8d0bce546fae85a9a146da8970877cc00e/docs/demos/demo_settings/saved_model_from_demo.json"

with urllib.request.urlopen(url) as u:
    settings = json.loads(u.read().decode())

# # TODO: Fix this, shouldn't be needed
for r in settings["blocking_rules_to_generate_predictions"]:
    del r["sql_dialect"]

settings["blocking_rules_to_generate_predictions"].append(block_on("city", salting_partitions=10))


linker = Linker(df, "../demo_settings/saved_model_from_demo.json", database_api=DuckDBAPI())
df_predictions = linker.inference.predict(threshold_match_probability=0.1)




## Load in labels

The labels file contains a list of pairwise comparisons which represent matches and non-matches.

The required format of the labels file is described [here](https://moj-analytical-services.github.io/splink/linkerqa.html#splink.linker.Linker.roc_chart_from_labels).


In [4]:
from splink.datasets import splink_dataset_labels

df_labels = splink_dataset_labels.fake_1000_labels
labels_table = linker.table_management.register_labels_table(df_labels)
df_labels.head(5)

,unique_id_l,source_dataset_l,unique_id_r,source_dataset_r,clerical_match_score
0,0,fake_1000,1,fake_1000,1.0
1,0,fake_1000,2,fake_1000,1.0
2,0,fake_1000,3,fake_1000,1.0
3,0,fake_1000,4,fake_1000,0.0
4,0,fake_1000,5,fake_1000,0.0


### Threshold Selection chart

Splink includes an interactive dashboard that shows key accuracy statistics:

In [5]:
linker.evaluation.accuracy_analysis_from_labels_table(
    labels_table, output_type="threshold_selection", add_metrics=["f1"]
)

alt.HConcatChart(...)

## Receiver operating characteristic curve

A [ROC chart](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) shows how the number of false positives and false negatives varies depending on the match threshold chosen. The match threshold is the match weight chosen as a cutoff for which pairwise comparisons to accept as matches.


In [6]:
linker.evaluation.accuracy_analysis_from_labels_table(
    labels_table, output_type="roc"
)

alt.Chart(...)

### Precision-recall chart

An alternative representation of truth space is called a [precision recall curve](https://stats.stackexchange.com/questions/7207/roc-vs-precision-and-recall-curves).

This can be plotted as follows:


In [7]:
linker.evaluation.accuracy_analysis_from_labels_table(
    labels_table, output_type="precision_recall"
)

alt.Chart(...)

## Truth table

Finally, Splink can also report the underlying table used to construct the ROC and precision recall curves.


In [8]:
roc_table = linker.evaluation.accuracy_analysis_from_labels_table(labels_table, output_type="table")
roc_table.as_pandas_dataframe(limit=5)

,truth_threshold,match_probability,total_clerical_labels,p,n,tp,tn,fp,fn,P_rate,N_rate,tp_rate,tn_rate,fp_rate,fn_rate,precision,recall,specificity,npv,accuracy,f1,f2,f0_5,p4,phi
0,-12.4,0.000185,3176.0,2031.0,1145.0,1027.0,1145.0,0.0,1004.0,0.639484,0.360516,0.505662,1.0,0.0,0.494338,1.0,0.505662,1.0,0.532806,0.683879,0.671681,0.561141,0.836455,0.683240,0.519057
1,-6.1,0.014369,3176.0,2031.0,1145.0,1026.0,1145.0,0.0,1005.0,0.639484,0.360516,0.505170,1.0,0.0,0.494830,1.0,0.505170,1.0,0.532558,0.683564,0.671246,0.560656,0.836186,0.682913,0.518683
2,-5.0,0.030303,3176.0,2031.0,1145.0,1024.0,1145.0,0.0,1007.0,0.639484,0.360516,0.504185,1.0,0.0,0.495815,1.0,0.504185,1.0,0.532063,0.682935,0.670376,0.559685,0.835646,0.682259,0.517937
3,-4.9,0.032407,3176.0,2031.0,1145.0,1019.0,1145.0,0.0,1012.0,0.639484,0.360516,0.501723,1.0,0.0,0.498277,1.0,0.501723,1.0,0.530830,0.681360,0.668197,0.557257,0.834289,0.680622,0.516071
4,-4.6,0.039602,3176.0,2031.0,1145.0,1018.0,1145.0,0.0,1013.0,0.639484,0.360516,0.501231,1.0,0.0,0.498769,1.0,0.501231,1.0,0.530584,0.681045,0.667760,0.556771,0.834016,0.680295,0.515699


!!! note "Further Reading"
:material-tools: For more on the quality assurance tools in Splink, please refer to the [Evaluation API documentation](../../linkereval.md).

    :bar_chart: For more on the charts used in this tutorial, please refer to the [Charts Gallery](../../charts/index.md#model-evaluation).

    :material-thumbs-up-down: For more on the Evaluation Metrics used in this tutorial, please refer to the [Edge Metrics guide.](../../topic_guides/evaluation/edge_metrics.md)


## :material-flag-checkered: That's it!

That wraps up the Splink tutorial! Don't worry, there are still plenty of resources to help on the next steps of your Splink journey:

:octicons-link-16: For some end-to-end notebooks of Splink pipelines, check out our [Examples](../examples/examples_index.md)

:simple-readme: For more deepdives into the different aspects of Splink, and record linkage more generally, check out our [Topic Guides](../../topic_guides/topic_guides_index.md)

:material-tools: For a reference on all the functionality avalable in Splink, see our [Documentation](../../documentation_index.md)
